In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install vaderSentiment

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.util import ngrams
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
def external_features(dataset_loc,):  
  english_dataset = pd.read_csv(dataset_loc)
  headline = english_dataset['Headline']
  body = english_dataset['Body']
	
  eng_ext = []
  i = 0
  for sent1,sent2 in zip(headline,body):
    print(i)
    i+=1
    vec = []

    #character ngrams
    for n in range(2,17):
      n_grams_1 = ngrams(sent1.lower(), n)
      n_grams_2 = ngrams(sent2.lower(),n)
      vec.append(len(list(set(n_grams_1).intersection(set(n_grams_2)))))
      temp_c1=0
      temp_c2=0
      n_grams_1 = ngrams(sent1.lower(), n)
      n_grams_3 = ngrams(sent2.lower()[:255],n)
      temp_c1 = len(list(set(n_grams_1).intersection(set(n_grams_3))))
      n_grams_1 = ngrams(sent1.lower(), n)
      n_grams_4 = ngrams(sent2.lower()[:100],n)
      temp_c2 = len(list(set(n_grams_1).intersection(set(n_grams_4))))
      # for grams in n_grams_1:
      #   if grams in ngrams(sent2[:255].lower(),n):
      #     temp_c+=1
      #   if grams in ngrams(sent2[:100].lower(),n):
      #     temp_c1+=1
      #print(temp_c1,temp_c2)
      vec.append(temp_c1)
      vec.append(temp_c2)

    #word ngrams
    for n in range(2,7):
      n_grams_1 = ngrams(sent1.lower().split(), n)
      n_grams_2 = ngrams(sent2.lower().split(),n)
      vec.append(len(list(set(n_grams_1).intersection(set(n_grams_2)))))
      temp_c=0
      n_grams_1 = ngrams(sent1.lower().split(), n)
      n_grams_3 = ngrams(sent2.lower()[:255].split(),n)
      temp_c=len(list(set(n_grams_1).intersection(set(n_grams_3))))
      # for grams in n_grams_1:
      #   if grams in sent2[:255]:
      #     temp_c+=1
      # print(vec[-1],temp_c)
      vec.append(temp_c)

    #no of common words between headline and body with respect to total words
    s1 = sent1.split()
    s2 = sent2.split()
    vec.append(len(set(s1).intersection(s2)) / float(len(set(s1).union(s2))))

    sid_obj = SentimentIntensityAnalyzer()
    d1 = sid_obj.polarity_scores(sent1)
    d2 = sid_obj.polarity_scores(sent2)
    vec.append(np.absolute(d1['neg']-d2['neg']))
    vec.append(np.absolute(d1['neu']-d2['neu']))
    vec.append(np.absolute(d1['pos']-d2['pos']))
    vec.append(np.absolute(d1['compound']-d2['compound']))

    eng_ext.append(vec)

  eng_ext = np.array(eng_ext)
  return eng_ext


In [ ]:
train_external_features = external_features('gdrive/MyDrive/CS626/Project/Data/train_Set.csv')

In [ ]:
train_external_features.shape

In [ ]:
train_external_features[15]

In [ ]:
np.save(arr=train_external_features,file='gdrive/MyDrive/CS626/Project/Data/train_external_features.npy')

In [ ]:
test_external_features = external_features('gdrive/MyDrive/CS626/Project/Data/test_Set.csv')

In [ ]:
test_external_features.shape

In [ ]:
test_external_features[19:21]

In [ ]:
np.save(arr=test_external_features,file='gdrive/MyDrive/CS626/Project/Data/test_external_features.npy')